# Consistent Bayes: Some Motivating Examples
---

Copyright 2017-2018 Michael Pilosov


### Import Libraries
_tested with python 3.6 on 01/26/18_

In [1]:
# Mathematics and Plotting
from HelperFuns import * # pyplot wrapper functions useful for visualizations, numpy, scipy, etc.
%matplotlib inline
plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = 10, 5
from cbayes import sample, solve, distributions
# Interactivity
from ipywidgets import *

---

Some introductory text goes here.   
Define $\Lambda$, $\mathcal{D}$.


--- 

## Sample from $\Lambda$
_Here we implement uniform random priors on the unit hypercube_

In [2]:
input_dim = 2 # Specify input space dimension (n)
num_samples = int(1E4) # number of input samples (N)

s_set = sample.sample_set(size=(num_samples, input_dim))
s_set.set_dist('uniform', {'loc': 0, 'scale': 1})
s_set.generate_samples()

lam = s_set.samples # create a pointer for ease of reference later with plotting.

---
## Define Parameter to Observables (PtO) Map
_ Choose from one of the following example options, feel free to add your own _ 

$O_1(\lambda) = \sum_{i=1}^n \lambda_i$  

$O_2(\lambda) = \lbrace \lambda_0 - \lambda_1, \; \;\lambda_1\rbrace$ 


In [3]:
PtO_fun_choice = 1

def fun1(lam): # sum all params
    return np.sum(lam,axis=1)

def fun2(lam): # pull two params, linear combination.
    return np.array([ lam[:,0] - lam[:,1], lam[:,1] ])

if PtO_fun_choice == 1:
    PtO_fun = fun1
elif PtO_fun_choice == 2:
    PtO_fun = fun2
else:
    raise( ValueError('Specify Proper PtO choice!') )

---
## Compute Data Space $O(\Lambda) = \mathcal{D}$ 

Format: `(n_dims, n_samples)`  
_Optional_: Specify subset of PtO map's components to use for inversion using the variable `sub_indices` 

In [4]:
p_set = sample.map_samples_and_create_problem(s_set, PtO_fun)
D_full = p_set.output.samples

sub_indices = None
if sub_indices is not None:
    D = D_full[:,sub_indices]
else:
    D = D_full
    type(np.sum(lam, axis=1))
# this is how we handle exceptions:
try:
    output_dim = D.shape[1]
except IndexError:
    output_dim = 1
print('dimensions :  lambda = '+str(lam.shape)+'   D = '+str(D.shape)+'   D_full = '+str(D_full.shape) )

dimensions :  lambda = (10000, 2)   D = (10000, 1)   D_full = (10000, 1)


## Compute Push-Forward of the Prior $P_{O(\Lambda)}$
_ ... i.e. Visualize the Data Space_

In [5]:
# Interactive Marginal Visualization
p_set.compute_pushforward_dist()
pf_dens = p_set.pushforward_dist

In [6]:
# Can plot "slices" of densities to observe differences between posterior and prior, but not that useful
data_min, data_max = -.25,4.25 # linspace parameters for plotting
plot_grid = np.linspace(data_min, data_max, 100)
widgets.interact(view_est_dens, x = fixed(plot_grid), 
         estimated_dens = fixed(pf_dens), 
         lab = fixed('KDE data'), title=fixed('Pushforward of Prior'),
         viewdim=(0, output_dim-1, 1) )
plt.show()

interactive(children=(IntSlider(value=0, description='viewdim', max=0), Output()), _dom_classes=('widget-interact',))

## Define Observed Probability Measure $P_\mathcal{D}$

In [7]:
# p_set.set_observed_dist('normal', {'loc':[0.5, 0.4], 'scale':0.1}) # better for function choice = 2
p_set.set_observed_dist('uni', {'loc':1, 'scale':0.25}) # default is normal based on the data space # for function choice = 1

obs_dist = p_set.observed_dist # this is define a pointer for ease of reference.

widgets.interact(view_analytical_dens, x = fixed(plot_grid), 
         analytical_dens = fixed(obs_dist), 
         lab = fixed('Observed'), title=fixed('Observed $P_\mathcal{D}$'),
         viewdim=(0, output_dim-1, 1) )
plt.show()

interactive(children=(IntSlider(value=0, description='viewdim', max=0), Output()), _dom_classes=('widget-interact',))

---

At this point we have performed the computations we need to. We have evaluated the input points through our map and performed a KDE on them. It would be useful at this point to save this object and/or its evaluation at every point in the data space for later re-use. Doing so here would be an appropriate place. 

--- 

## Accept/Reject Sampling of Posterior

Since we have already used the samples in our prior to compute the pushforward density, we can re-use these with an accept/reject algorithm to get a set of samples generated from the posterior according to the solution of the stochastic inverse problem as outlined in the Consistent Bayes formulation. 

In [8]:
p_set.set_ratio()
eta_r = p_set.ratio
solve.problem(p_set)

In [9]:
accept_inds = p_set.accept_inds
lam_accept = p_set.input.samples[accept_inds,:]
num_accept = len(accept_inds)
print('Number accepted: %d = %2.2f%%'%(num_accept, 100*np.float(num_accept)/num_samples))

Number accepted: 1868 = 18.68%


### Visualize Accept/Reject Samples

In [10]:
widgets.interact_manual(pltaccept, lam = fixed(lam), inds = fixed(accept_inds), 
         N = (1, num_accept+1, 10), eta_r = fixed(eta_r), 
         i = (0, input_dim-1, 1), j = (0, input_dim-1, 1))
# You will visualize the accepted samples in a subset of size N of the input samples. 
# This is for faster plotting but also so you can see the progression of accepted sampling in the algorithm.


interactive(children=(IntSlider(value=931, description='N', max=1869, min=1, step=10), IntSlider(value=0, description='i', max=1), IntSlider(value=1, description='j', max=1), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function HelperFuns.pltaccept>

---
## Visualize Posterior Density

In [11]:
prior_dens_kde = distributions.gkde(lam) # this method wants dimensions different than ours.
post_dens_kde = distributions.gkde(lam_accept)

lam_min, lam_max = p_set.input.dist.a, p_set.input.dist.b # linspace parameters for plotting
lam_plot_grid = np.linspace(lam_min, lam_max, 100)

# Can plot "slices" of densities to observe differences between posterior and prior, but not that useful
interact(compare_est_input_dens, x = fixed(lam_plot_grid), 
         estimated_dens1 = fixed(prior_dens_kde), estimated_dens2 = fixed(post_dens_kde), 
         lab_1 = fixed('KDE prior'), lab_2 = fixed('KDE post'), title=fixed(''),
         viewdim=(0, input_dim-1, 1))
plt.show()

interactive(children=(IntSlider(value=0, description='viewdim', max=1), Output()), _dom_classes=('widget-interact',))

## Visualize Quality of Solution 
_We compare the push-forward of the posterior using accepted samples against the observed density_

In [12]:
push_post_dens_kde = distributions.gkde(D[accept_inds,:])
# diagonal crossection view
interact(compare_output_dens, x = fixed(plot_grid), 
         analytical_dens = fixed(obs_dist), estimated_dens = fixed(push_post_dens_kde), 
         lab_1 = fixed('observed'), lab_2 = fixed('KDE push'), title = fixed('Slice - Observed v. PF'),
         viewdim = (0, output_dim-1, 1))
plt.show()

interactive(children=(IntSlider(value=0, description='viewdim', max=0), Output()), _dom_classes=('widget-interact',))